# Walkthru 12


## References
- [Walkthru 12](https://youtu.be/GuCkpjXHdTc)

In [1]:
from fastai.vision.all import *
import timm

In [2]:
path = Path('paddy-disease-classification')

train_path = path / 'train_images'
train_files = get_image_files(train_path)

test_path = path/'test_images'
test_files = get_image_files(train_path).sorted()

In [3]:
! ls {train_path}

bacterial_leaf_blight	  blast       downy_mildew  tungro
bacterial_leaf_streak	  brown_spot  hispa
bacterial_panicle_blight  dead_heart  normal


In [4]:
## For walkthru 12

def custom_train(epochs, lr, arch, item_tfms, batch_tfms, seed=None, accum=False, floating=16):
    kwargs = {'bs': 32} if accum else {}
    
    dls = ImageDataLoaders.from_folder(
        train_path,
        seed=seed,
        valid_pct=0.2,
        item_tfms=item_tfms,
        batch_tfms=batch_tfms,
        **kwargs
    )
    cbs = GradientAccumulation(2) if accum else []
    
    if floating == 16:
        learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
    else:
        learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs)
        
    learn.fine_tune(epochs, lr)
    
    return learn

In [5]:
arch = 'convnext_tiny_in22k'

item_tfms = Resize(480)
batch_tfms = aug_transforms(size=224, min_scale=0.75)


In [6]:
#! nvidia-smi --query-gpu=timestamp,power.draw,memory.total,utilization.memory,temperature.gpu,clocks.current.graphics,clocks.current.sm,clocks.current.memory --loop=5 --format=csv --filename=results-file.csv

/home/fmussari/.cache/torch/hub/checkpoints/

In [7]:
learn = custom_train(3, 0.02, arch, item_tfms, batch_tfms, seed=42, accum=True, floating=16)

epoch,train_loss,valid_loss,error_rate,time
0,1.426843,1.159889,0.298414,03:29


epoch,train_loss,valid_loss,error_rate,time
0,0.831079,0.591243,0.181643,03:57
1,0.396642,0.260589,0.079289,04:00
2,0.219774,0.162865,0.050457,03:59


In [8]:
from copy import deepcopy

In [9]:
learn.export('walk12-convnext_tiny_in22k-3ep.pkl')

In [11]:
m = learn.model
m

Sequential(
  (0): TimmBody(
    (model): ConvNeXt(
      (stem): Sequential(
        (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
        (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
      )
      (stages): Sequential(
        (0): ConvNeXtStage(
          (downsample): Identity()
          (blocks): Sequential(
            (0): ConvNeXtBlock(
              (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
              (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
              (mlp): Mlp(
                (fc1): Linear(in_features=96, out_features=384, bias=True)
                (act): GELU()
                (drop1): Dropout(p=0.0, inplace=False)
                (fc2): Linear(in_features=384, out_features=96, bias=True)
                (drop2): Dropout(p=0.0, inplace=False)
              )
              (drop_path): Identity()
            )
            (1): ConvNeXtBlock(
              (conv_dw): Conv2d(96, 

In [12]:
learn.summary()

Sequential (Input shape: 32 x 3 x 224 x 224)
Layer (type)         Output Shape         Param #    Trainable 
                     32 x 96 x 56 x 56   
Conv2d                                    4704       True      
LayerNorm2d                               192        True      
Identity                                                       
Conv2d                                    4800       True      
LayerNorm                                 192        True      
____________________________________________________________________________
                     32 x 56 x 56 x 384  
Linear                                    37248      True      
GELU                                                           
Dropout                                                        
____________________________________________________________________________
                     32 x 56 x 56 x 96   
Linear                                    36960      True      
Dropout                            

In [13]:
h = m[1]

In [14]:
h

Sequential(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten(full=False)
  (2): BatchNorm1d(1536, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=1536, out_features=512, bias=False)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=10, bias=False)
)

In [15]:
h[-1]

Linear(in_features=512, out_features=10, bias=False)

In [20]:
curr_loss = learn2.loss_func

In [21]:
def dtc_loss(preds, targs):
    rice_preds, dis_preds = preds
    return curr_loss(dis_preds, targs)

In [16]:
class DiseaseAndTypeClassifier(nn.Module):
    
    def __init__(self, m):
        super().__init__()
        self.l1 = nn.Linear(512, 10, bias=False) # Rice Type
        self.l2 = nn.Linear(512, 10, bias=False) # Disease
        del(m[1][-1])
        self.m = m
    
    def forward(self, x):
        x = self.m(x)
        x1 = self.l1(x)
        x2 = self.l2(x)
        return x1, x2

In [17]:
learn2 = deepcopy(learn)
dtc = DiseaseAndTypeClassifier(learn2.model)


In [18]:
learn2.model = dtc
learn2.loss_func = dtc_loss

In [19]:
preds, targs = learn2.get_preds(dl=learn2.dls.valid)

AttributeError: 'tuple' object has no attribute 'view'

Avoid metrics for now

In [23]:
dls = ImageDataLoaders.from_folder(
    train_path,
    seed=42,
    valid_pct=0.2,
    item_tfms=item_tfms,
    batch_tfms=batch_tfms,
)

In [24]:
learn = vision_learner(dls, arch)
learn2 = deepcopy(learn)

In [25]:
dtc = DiseaseAndTypeClassifier(learn2.model)

In [26]:
learn2.model = dtc
learn2.loss_func = dtc_loss

In [27]:
preds, targs = learn2.get_preds(dl=learn2.dls.valid)

In [29]:
rice_preds, dis_preds = preds